In [ ]:
import json
import requests
import datetime
import pandas as pd

catalog = input("Please enter catalog number: ")
url = f'https://auctionsplus.com.au/auctionmanagement/api/catalogue/{catalog}'
data = requests.get(url)
if data.status_code == 200:
    auctions = json.loads(data.content)
#     with open(f"auction_{auctions['auctionId']}.json", 'w') as f:
#         json.dump(auctions, f)

#print(f"Lot, Description, Head, Live wgt, Dressed wgt, Age, StartPrice, Reserve, Reserve/dressed, Town, RegionName")
lot_data = {}
for mob in auctions['assessedLots']:
    if mob['reservePrice']:
        if not mob['age']['high'] and not mob['age']['low']:
            if mob['dropDetail']['earliestDrop']['day']:
                earliest = datetime.datetime(year=mob['dropDetail']['earliestDrop']['year'],
                              month=mob['dropDetail']['earliestDrop']['month'],
                              day=mob['dropDetail']['earliestDrop']['day'])
            else:
                earliest = datetime.datetime(year=mob['dropDetail']['earliestDrop']['year'],
                                             month=mob['dropDetail']['earliestDrop']['month'], day=1)
            if mob['dropDetail']['latestDrop']['day']:
                latest = datetime.datetime(year=mob['dropDetail']['latestDrop']['year'],
                              month=mob['dropDetail']['latestDrop']['month'],
                              day=mob['dropDetail']['latestDrop']['day'])
            else:
                latest = datetime.datetime(year=mob['dropDetail']['latestDrop']['year'],
                              month=mob['dropDetail']['latestDrop']['month'], day=1)
            age = (datetime.datetime.now() - earliest).days / 365
        else:
            age_hi = mob['age']['high']
            age_lo = mob['age']['low']
            if mob['age']['rangeDescription'] == 'Months':
                divisor = 12
            else:
                divisor = 1
            if age_hi and age_lo:
                age = (age_hi + age_lo) / 2 / divisor
            elif age_hi and not age_lo:
                age = age_hi / divisor
            elif age_lo and not age_hi:
                age = age_lo / divisor
            else:
                age = 'Unknown'
        live_avg = mob['deliveryLiveWeight']['avg']
        dressed_avg = mob['deliveryDressedWeight']['avg']
        res_price = mob['reservePrice']
        lot_data[mob['lotIdentifier']] = [mob['lotBestDescription'], mob['numberOfHead'], live_avg, dressed_avg, age,
                                          mob['startPrice'], res_price, res_price / dressed_avg,
                                          mob['deliveryLocation']['townName'], mob['deliveryLocation']['regionName']]

lots = pd.DataFrame.from_dict(lot_data, orient='index',
                              columns=["Description", "Head", "Live wgt", "Dressed wgt", "Age", "StartPrice",
                                       "Reserve", "Reserve/dressed", "Town", "RegionName"])
from IPython.display import display, HTML

display(HTML(lots.to_html()))